# Generative AI

## Financial Analysis Generation

Sell side analyst needs to write analysis report regularly, and there are some template that they use. following is a sample report on Apple Q4 2020 about its quarterly financial report. 

```
Apple Q4 2020 Results Exceed Expectations; Raising FVE to $145

Financial Summary:
Apple reported stellar fiscal Q1 2021 results with revenue of $111.4 billion, up 21% year-over-year, easily beating consensus estimates. All product categories saw double-digit growth led by iPhone (up 17% to $65.6B), Services (up 24% to $15.8B), and Wearables/Home/Accessories (up 30% to $13.0B).

Recommendation:
We reiterate our Buy rating on Apple shares. The stock is currently trading at $137, providing an upside of over 15% from current levels based on our revised $145 fair value estimate.

Risk Factors:
- Potential supply chain disruptions if COVID-19 situation worsens
- Intense competition, especially in 5G smartphones
- Regulatory scrutiny around App Store policies

In [42]:
# Please update with your own deployed endpoint name and model name. 
endpoint_name = 'jumpstart-dft-hf-llm-mistral-7b-ins-xxx'
model_name='huggingface-llm-mistral-7b-instruct-xxx'

In [34]:
from IPython.display import HTML

pdf_path = 'reports/Apple-FY21-Q1-Financial-Statement.pdf'
html = f'<iframe src="{pdf_path}" width="100%" height="600px"></iframe>'
HTML(html)


/opt/conda/lib/python3.10/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [15]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 70.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 88.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [46]:
import pdfplumber

def read_pdf(file_name):
    # Open the PDF file
    with pdfplumber.open(file_name) as pdf:
        # Initialize an empty string to store the text
        text = ''
    
        # Loop through each page and extract the text
        for page in pdf.pages:
            text += page.extract_text()
    return text

GenAI might be able to help with prompt below.

In [39]:
import json
import boto3
newline, bold, unbold = '\n', '\033[1m', '\033[0m'

def query_endpoint(payload):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, 
        ContentType='application/json', 
        Body=json.dumps(payload).encode('utf-8'),
        InferenceComponentName=model_name
    )
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions[0]['generated_text']
    print (
        f"Generated Text: {bold}{generated_text}{unbold}{newline}")
    # print (
    #     f"Input Text: {payload['inputs']}{newline}"
    #     f"Generated Text: {bold}{generated_text}{unbold}{newline}")

In [47]:
financial_report_text = read_pdf('reports/Apple-FY21-Q1-Financial-Statement.pdf')

prompt = f"""
You are a financial analyst tasked with analyzing a company's financial report and generating an analyst report.

Instructions:
1. Carefully read and analyze the PDF financial report, paying close attention to key financial statements, ratios, and other relevant information.

<financial_report>
{financial_report_text}
</financial_report>

2. Generate an analyst report with the following sections:

Financial Report Summary:
- Provide a concise summary of the company's financial performance, highlighting key metrics, trends, and notable figures from the financial statements.
- Discuss the company's revenue, profitability, cash flow, and balance sheet strength.

Buy/Sell Recommendation:
- Based on your analysis, provide a clear recommendation on whether an investor should buy, sell, or hold the company's stock.
- You estimated fair value (FVE), compare it with current apple share price is $130.
- Support your recommendation with specific evidence and reasoning from the financial report analysis.

Risk Alert/Warning:
- Identify and discuss any significant risks, concerns, or potential red flags that emerged from your analysis of the financial report.
- Explain the potential impact of these risks on the company's future performance and valuation.

3. Ensure that your analyst report is well-structured, easy to understand, and provides actionable insights for investors and stakeholders.

4. Double-check your analysis and recommendations for accuracy and consistency with the information provided in the financial report.

Please let me know if you need any clarification or have additional requirements for the analyst report.

"""

payload = {
    "inputs": prompt,
    "parameters":{
        "max_new_tokens": 2000,
        "return_full_text": False,
        "do_sample": True,
        "top_k":10
        }
}


In [33]:
query_endpoint(payload)

Generated Text: ---

**Analyst Report**

**Apple Inc. (AAPL): Q4 2020 Financial Analysis**

**Financial Report Summary:**

Apple Inc. reported strong financial results for the quarter ended December 26, 2020. The company generated net sales of $111.4 billion, a notable increase of 21.2% year-over-year. Apple's profitability remains robust, as evidenced by its operating income of $33.6 billion, a growth of 31.2% compared to Q4 2019.

Key revenue contributors for Apple include its Products segment, which accounted for 86.6% of the total net sales, generating $95.7 billion. The Services segment reported sales of $15.8 billion, a 16.3% increase YoY, indicating a continued focus on expanding its services business.

The cash flow statement reveals that Apple generated $38.8 billion in cash from operating activities, up from $30.5 billion in the same quarter last year. The company's cash position improved, with a cash balance of $37.7 billion, compared to $41.7 billion in the previous quarter

You can re-run and modify until you get a satisfactory version. In future, you can use the version as a template in few shot prompt. For example, in the next year, Apple released Apple Q4 2021 quarterly financial report, you can use this template to generate new report.

In [52]:
analysis_template = """
**Analyst Report**

**Apple Inc. (AAPL): Q4 2020 Financial Analysis**

**Financial Report Summary:**

Apple Inc. reported strong financial results for the quarter ended December 26, 2020. The company generated net sales of $111.4 billion, a notable increase of 21.2% year-over-year. Apple's profitability remains robust, as evidenced by its operating income of $33.6 billion, a growth of 31.2% compared to Q4 2019.

Key revenue contributors for Apple include its Products segment, which accounted for 86.6% of the total net sales, generating $95.7 billion. The Services segment reported sales of $15.8 billion, a 16.3% increase YoY, indicating a continued focus on expanding its services business.

The cash flow statement reveals that Apple generated $38.8 billion in cash from operating activities, up from $30.5 billion in the same quarter last year. The company's cash position improved, with a cash balance of $37.7 billion, compared to $41.7 billion in the previous quarter.

Apple's balance sheet exhibits strong liquidity and solvency, as evidenced by a current ratio of 1.15 and a quick ratio of 0.93. The company's debt position has increased slightly, with total debt increasing to $290.2 billion from $282.6 billion in the previous quarter.

**Buy/Sell Recommendation:**

Based on the analysis of Apple's Q4 2020 financial report, I would recommend a strong buy position for investors considering AAPL. The company's continued revenue growth and strong profitability make it an attractive investment. Our Fair Value Estimate (FVE) for Apple shares is $140, and given the current share price ($130), we believe there is a solid upside potential.

The Services segment's growth is particularly noteworthy. Apple's growing services business is a significant value driver, providing recurring revenue and contributing to the company's overall profitability. In combination with the robust sales of its hardware products, Apple's services business sets it apart from its competitors and positions the company well to capitalize on emerging trends, such as the increasing demand for remote work solutions.

**Risk Alert/Warning:**

Despite the positive outlook, there are some potential risks and concerns that investors should be aware of when considering an investment in Apple. The ongoing global economic uncertainty due to the COVID-19 pandemic and its impact on consumer spending is a primary concern. Additionally, the intensifying competition, particularly in the smartphone market, could put downward pressure on Apple's sales and profitability. Lastly, the regulatory environment in major markets, such as the US and China, could introduce uncertainty and impact Apple's operations and financial performance.

While these risks may pose a threat to Apple's future performance, the company's strong financial position, innovative product offerings, and strategic focus on services provide solid foundation for managing these challenges. We believe that Apple's proven track record of resilience and adaptability position it well to navigate any potential headwinds.
"""

financial_report_text = read_pdf('reports/Apple_FY22_Q1_Consolidated_Financial_Statements.pdf')

prompt = f"""
You are a financial analyst tasked with analyzing a company's financial report and generating an analyst report. 

Instructions:
1. Carefully read and analyze the PDF financial report, paying close attention to key financial statements, ratios, and other relevant information.

<financial_report>
{financial_report_text}
</financial_report>

2. Generate an analyst report following the similar report template and format as below:

<analysis_template>
{analysis_template}
</analysis_template>

3. Ensure that your analyst report is well-structured, easy to understand, and follow the template structure.

4. Double-check your analysis and recommendations for accuracy and consistency with the information provided in the financial report.

Please let me know if you need any clarification or have additional requirements for the analyst report.

"""

payload = {
    "inputs": prompt,
    "parameters":{
        "max_new_tokens": 2000,
        "return_full_text": False,
        "do_sample": True,
        "top_k":10
        }
}


In [51]:
query_endpoint(payload)

Generated Text: <analysis>
**Analyst Report**

**Apple Inc. (AAPL): Q4 2021 Financial Analysis**

**Financial Report Summary:**

Apple Inc. reported impressive financial results for the quarter ended December 25, 2021. The company generated net sales of $123.9 billion, a significant increase of 11.3% year-over-year. Apple's profitability remained solid, as shown by its operating income of $41.3 billion, representing a respectable 27.1% increase compared to Q4 2020.

The company's key revenue drivers include the Products segment, which accounted for 91.8% of Apple's total net sales, generating $111.7 billion. The Services segment reported sales of $12.2 billion, a strong growth of 37.2% YoY, highlighting the importance of its services business.

A deeper look at Apple's cash flow statement reveals that the company generated $46.9 billion in cash from operating activities, representing a 23% increase year-over-year. The company's cash position strengthened, with a cash balance of $38.6 b